In [1]:
# Intialization
import os
import time

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

# spark imports
from pyspark.sql import SparkSession

# data science imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import unicodedata

In [ ]:
data_path = 'hdfs:///user/andrew/'

In [2]:
# Read in data through spark since the data is sored in hadoop and format the columns
# Convert to pandas dataframes for easier and faster manipulation
from pyspark.sql.types import *
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

# Rating = Row("user_id", "item_id", "label") # Ignore timestamp
# User = Row("user_id", "gender", "age_group" ,"occupation")

ratings = sqlContext.read.parquet(data_path + 'ratings_20m')
ratings_df = ratings.toPandas()
ratings_df.drop(columns = ['timestamp'], inplace = True)
ratings_df.columns = ['user_id', 'item_id', 'label']
ratings_df['user_id'] = ratings_df.user_id.apply(lambda x: 'user_id_' + str(x))
ratings_df['item_id'] = ratings_df.item_id.apply(lambda x: 'item_id_' + str(x))

In [3]:
movies_metadata = sqlContext.read.parquet(data_path + 'movie_20m_metadata_original')
movies_metadata_df = movies_metadata.toPandas()

movies_metadata_df.genres = movies_metadata_df.genres.apply(lambda x: x.split(', '))
movies_metadata_df.imdb_genres = movies_metadata_df.imdb_genres.apply(lambda x: x.split(', '))
movies_metadata_df.director = movies_metadata_df.director.apply(lambda x: x.split(', '))
movies_metadata_df.actors = movies_metadata_df.actors.apply(lambda x: x.split(', '))

In [4]:
# Binarize Movie Lens Genre Columns
mlb_movie_lens_genres = MultiLabelBinarizer()
movies_df = movies_metadata_df.join(
    pd.DataFrame(mlb_movie_lens_genres.fit_transform(movies_metadata_df.genres),
                 columns = mlb_movie_lens_genres.classes_,
                 index = movies_metadata_df.index).add_prefix('ml_genre_'))

# Binarize IMDb Genre Columns
mlb_imdb_genres = MultiLabelBinarizer()
movies_df = movies_df.join(pd.DataFrame(mlb_imdb_genres.fit_transform(movies_df.imdb_genres),
                                        columns = mlb_imdb_genres.classes_,
                                        index = movies_df.index).add_prefix('imdb_genre_')) 

# Binarize IMDb Director Columns
mlb_imdb_director = MultiLabelBinarizer()
movies_df = movies_df.join(pd.DataFrame(mlb_imdb_director.fit_transform(movies_df.director),
                                        columns = mlb_imdb_director.classes_,
                                        index = movies_df.index).add_prefix('director_')) 

# Binarize IMDb Actor Columns
mlb_imdb_actors = MultiLabelBinarizer()
movies_df = movies_df.join(pd.DataFrame(mlb_imdb_actors.fit_transform(movies_df.actors),
                                        columns = mlb_imdb_actors.classes_,
                                        index = movies_df.index).add_prefix('actor_'))

In [5]:
del ratings, movies_metadata, movies_metadata_df, \
    mlb_movie_lens_genres, mlb_imdb_genres, mlb_imdb_director, mlb_imdb_actors

In [6]:
ohe_mpaa_ratings = pd.get_dummies(movies_df.MPAA_rating, prefix = 'MPAA_rating')
movies_df = pd.concat([movies_df, ohe_mpaa_ratings], axis = 1, sort = False)
del ohe_mpaa_ratings
movies_df.drop(['genres', 'imdb_genres', 'director', 'actors', 'MPAA_rating'], axis = 1, inplace = True)
movies_df.columns = [c.replace(' ', '_') for c in movies_df.columns]
movies_df.columns = [c.replace('/', '') for c in movies_df.columns]
movies_df.columns = [c.replace('.', '') for c in movies_df.columns]
movies_df.columns = [c.replace('-', '') for c in movies_df.columns]
movies_df.columns = [c.replace("'", '') for c in movies_df.columns]
movies_df.title = [unicodedata.normalize('NFKD', t).encode('ascii', 'ignore') 
                   for t in movies_df.title]

movies_df['movieId'] = movies_df.movieId.astype(int)
movies_df['title'] = movies_df.title.astype(str)
movies_df['imdb_id'] = movies_df.imdb_id.astype(str)
movies_df['imdb_rating'] = movies_df.imdb_rating.astype(float)
movies_df['imdb_votes'] = movies_df.imdb_votes.astype(int)
movies_df['metascore'] = movies_df.metascore.astype(int)
movies_df['runtime'] = movies_df.runtime.astype(int)
movies_df['year'] = movies_df.year.astype(int)
# movies_df.iloc[:, 8:] = movies_df.iloc[:, 8:].astype(int)

In [7]:
# Combine MPAA NOT RATED, Not Rated, Unrated, and UNRATED
movies_df['MPAA_rating_Unrated'] = movies_df[[
    'MPAA_rating_Unrated', 'MPAA_rating_UNRATED', 'MPAA_rating_NOT_RATED', 
    'MPAA_rating_Not_Rated', 'MPAA_rating_NR']].apply(np.sum, axis = 1)
# Combine APPROVED, Approved, blank, NA, Passed, PASSED
movies_df['MPAA_rating_Other'] = movies_df[[
    'MPAA_rating_APPROVED', 'MPAA_rating_Approved', 'MPAA_rating_', 
    'MPAA_rating_NA', 'MPAA_rating_Passed', 'MPAA_rating_PASSED', 
    'MPAA_rating_(BANNED)', 'MPAA_rating_OPEN', 'MPAA_rating_S']].apply(np.sum, axis = 1)
# Combine M, GP, MPG, and PG - These are all the same, just a how the ratings evolved
movies_df['MPAA_rating_PG'] = movies_df[[
    'MPAA_rating_PG', 'MPAA_rating_GP', 'MPAA_rating_M',
    'MPAA_rating_MPG']].apply(np.sum, axis = 1)
# Drop now unnecessary columns
movies_df.drop(['MPAA_rating_UNRATED', 'MPAA_rating_NOT_RATED', 'MPAA_rating_Not_Rated', 
                'MPAA_rating_NR', 'MPAA_rating_APPROVED', 'MPAA_rating_Approved', 
                'MPAA_rating_', 'MPAA_rating_NA', 'MPAA_rating_Passed', 
                'MPAA_rating_PASSED', 'MPAA_rating_(BANNED)', 'MPAA_rating_OPEN', 
                'MPAA_rating_S', 'MPAA_rating_GP', 'MPAA_rating_M', 'MPAA_rating_MPG'], 
               axis = 1, inplace = True)

In [8]:
# Merge columns with duplicate names
actor_Chuck_D_col = movies_df[['actor_Chuck_D']].apply(np.sum, axis = 1)
actor_William_col = movies_df[['actor_William']].apply(np.sum, axis = 1)
movies_df.drop(columns = ['actor_Chuck_D', 'actor_William'], inplace = True)
movies_df['actor_Chuck_D'] = actor_Chuck_D_col
movies_df['actor_William'] = actor_William_col
del actor_Chuck_D_col, actor_William_col
# Drop useless column
movies_df.drop(columns = ['ml_genre_(no_genres_listed)'], inplace = True)

movies_df['item_id'] = movies_df.movieId.apply(lambda x: 'item_id_' + str(x))
movies_df.drop(columns = ['movieId'], inplace = True)
movies_df = movies_df.set_index(movies_df.item_id) # set index so no sorting errors occur.

In [9]:
movies_df.head(5)

,title,imdb_id,imdb_rating,imdb_votes,metascore,runtime,year,ml_genre_Action,ml_genre_Adventure,ml_genre_Animation,...,MPAA_rating_TVMA,MPAA_rating_TVPG,MPAA_rating_TVY,MPAA_rating_TVY7,MPAA_rating_Unrated,MPAA_rating_X,MPAA_rating_Other,actor_Chuck_D,actor_William,item_id
item_id,,,,,,,,,,,,,,,,,,,,,
item_id_110811,Dead in the Water (2002),tt0235333,4.8,745,0,90,2002,0,0,0,...,0,0,0,0,0,0,0,0,0,item_id_110811
item_id_110813,Girl on a Bicycle (2013),tt1686042,6.2,1964,32,101,2013,0,0,0,...,0,0,0,0,0,0,0,0,0,item_id_110813
item_id_110818,Which Way to the Front? (1970),tt0066564,4.1,1017,0,96,1970,0,0,0,...,0,0,0,0,0,0,0,0,0,item_id_110818
item_id_110820,"Liberation of L.B. Jones, The (1970)",tt0065979,7.0,513,0,102,1970,0,0,0,...,0,0,0,0,0,0,0,0,0,item_id_110820
item_id_110824,Robocroc (2013),tt3188560,3.0,865,0,77,2013,1,0,0,...,0,0,0,0,0,0,0,0,0,item_id_110824


In [10]:
ratings_df.head(5)

,user_id,item_id,label
0,user_id_130432,item_id_3003,5.0
1,user_id_130432,item_id_3006,5.0
2,user_id_130432,item_id_3010,4.0
3,user_id_130432,item_id_3052,5.0
4,user_id_130432,item_id_3055,5.0


In [11]:
import sys
print 'Ratings DF Memory Usage'
print sys.getsizeof(ratings_df)

print 'Movies DF Memory Usage'
print sys.getsizeof(movies_df)

Ratings DF Memory Usage
2460103337
Movies DF Memory Usage
12619667794


In [12]:
import os
tot_m, used_m, free_m = map(int, os.popen('free -t -m').readlines()[-1].split()[1:])
print 'Total memory:', tot_m
print 'Used memory:', used_m
print 'Free memory:', free_m

Total memory: 517864
Used memory: 29837
Free memory: 486885


Find the unique set of movies that have ratings in ratings_df. Remove all movies without ratings from movies_df.
There are 3883 movies total, and only 3706 movies with ratings.

In [13]:
print len(ratings_df.item_id.unique())
print len(movies_df.item_id.unique())
#26744 < 27278 so some movies do not have any ratings, these movies can be removed for genre rating averaging
# keep only movies that have at least one user rating
distinct_rated_movies = ratings_df.item_id.unique()
movies_df_rated = movies_df.iloc[[item in distinct_rated_movies for item in movies_df.item_id]]
print(movies_df_rated.shape)
del distinct_rated_movies, movies_df

26744
27278
(26744, 57813)


In [15]:
tot_m, used_m, free_m = map(int, os.popen('free -t -m').readlines()[-1].split()[1:])
print 'Total memory:', tot_m
print 'Used memory:', used_m
print 'Free memory:', free_m

Total memory: 517864
Used memory: 41656
Free memory: 475066


Transform the ratings dataframe (user_id, item_id, label) to a user_id x item_id dataframe with the labels (ratings) as the values. Fill all missing values, items that have not been rated by a user, with 0s. 

Then create the binarized version of this matrix, and again fill any missing values with 0s. This dataframe will have values of 1 where a rating exists, and 0 otherwise.

In [16]:
# Transform ratings dataframe to user_id x item_id dataframe with label (or rating) as the values
ratings_spread = ratings_df.pivot(index='user_id',columns='item_id',values='label').fillna(0)
# Create binary valued dataframe from the transformed ratings dataframe. 
#   This should have values of 1 where the labels are >0 and 0 otherwise.
ratings_spread_binary = (ratings_spread/ratings_spread).fillna(0)

print('User Movie Ratings')
ratings_spread.head()

User Movie Ratings


item_id,item_id_1,item_id_10,item_id_100,item_id_1000,item_id_100003,item_id_100006,item_id_100008,item_id_100010,item_id_100013,item_id_100015,...,item_id_99962,item_id_99964,item_id_99968,item_id_99970,item_id_99986,item_id_99989,item_id_99992,item_id_99994,item_id_99996,item_id_99999
user_id,,,,,,,,,,,,,,,,,,,,,
user_id_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_10,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_10000,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Movie Genre Rating Summary

In [17]:
# Create movies dataframe subset that only contains the Movie Lens genre or IMDb genre.
genre_cols = [mg for mg in list(movies_df_rated.columns) if (mg[0:8] == 'ml_genre' or mg[0:10] == 'imdb_genre')]
movie_genres = movies_df_rated[genre_cols]
# Cross the new ratings dataframe with the genres dataframe 
#   - this is now a datframe with sum of ratings by genre for each user
user_genre_total = ratings_spread.dot(movie_genres)
# Cross the binarized ratings dataframe with the genres dataframe 
#   - this is now a dataframe that contains counts of rated movies by genre for each user
user_genre_count = ratings_spread_binary.dot(movie_genres)

In [18]:
ratings_spread_binary.head()

item_id,item_id_1,item_id_10,item_id_100,item_id_1000,item_id_100003,item_id_100006,item_id_100008,item_id_100010,item_id_100013,item_id_100015,...,item_id_99962,item_id_99964,item_id_99968,item_id_99970,item_id_99986,item_id_99989,item_id_99992,item_id_99994,item_id_99996,item_id_99999
user_id,,,,,,,,,,,,,,,,,,,,,
user_id_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_id_10000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
movie_genres.head()

,ml_genre_Action,ml_genre_Adventure,ml_genre_Animation,ml_genre_Children,ml_genre_Comedy,ml_genre_Crime,ml_genre_Documentary,ml_genre_Drama,ml_genre_Fantasy,ml_genre_FilmNoir,...,imdb_genre_News,imdb_genre_RealityTV,imdb_genre_Romance,imdb_genre_SciFi,imdb_genre_Short,imdb_genre_Sport,imdb_genre_TalkShow,imdb_genre_Thriller,imdb_genre_War,imdb_genre_Western
item_id,,,,,,,,,,,,,,,,,,,,,
item_id_110811,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
item_id_110813,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
item_id_110818,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
item_id_110824,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
item_id_110826,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Perform element wise division on user genre total and user genre count to get user genre average
user_genre_avg_ratings = user_genre_total / user_genre_count
user_genre_avg_ratings = user_genre_avg_ratings.add_suffix('_avg_rating').reset_index()

# users_metadata = pd.merge(users_df, user_genre_avg_ratings, on = 'user_id')

print('User Metadata Genre Averages')
# users_metadata.head()
user_genre_avg_ratings.head()

User Metadata Genre Averages


,user_id,ml_genre_Action_avg_rating,ml_genre_Adventure_avg_rating,ml_genre_Animation_avg_rating,ml_genre_Children_avg_rating,ml_genre_Comedy_avg_rating,ml_genre_Crime_avg_rating,ml_genre_Documentary_avg_rating,ml_genre_Drama_avg_rating,ml_genre_Fantasy_avg_rating,...,imdb_genre_News_avg_rating,imdb_genre_RealityTV_avg_rating,imdb_genre_Romance_avg_rating,imdb_genre_SciFi_avg_rating,imdb_genre_Short_avg_rating,imdb_genre_Sport_avg_rating,imdb_genre_TalkShow_avg_rating,imdb_genre_Thriller_avg_rating,imdb_genre_War_avg_rating,imdb_genre_Western_avg_rating
0,user_id_1,3.727273,3.787671,3.650000,3.605263,3.731707,3.809524,NaN,3.767442,3.789855,...,NaN,NaN,3.850000,3.709302,4.0,NaN,NaN,3.670732,3.666667,3.4
1,user_id_10,3.812500,3.909091,4.000000,4.000000,4.000000,4.250000,NaN,3.840000,4.000000,...,NaN,NaN,3.900000,3.800000,NaN,NaN,NaN,3.666667,3.800000,3.0
2,user_id_100,3.500000,3.500000,4.000000,2.500000,3.500000,3.800000,4.0,3.538462,3.666667,...,NaN,NaN,3.444444,3.636364,NaN,4.0,NaN,3.357143,3.000000,NaN
3,user_id_1000,4.328947,4.239130,4.100000,3.900000,3.500000,4.500000,NaN,4.500000,4.125000,...,NaN,NaN,4.500000,4.283333,NaN,NaN,NaN,4.275000,4.000000,NaN
4,user_id_10000,4.400000,4.705882,4.428571,4.307692,4.466667,4.250000,NaN,4.521739,3.875000,...,NaN,NaN,4.451613,3.833333,NaN,5.0,NaN,4.727273,3.000000,4.0


In [21]:
# del movie_genres, user_genre_total, user_genre_count, user_genre_avg_ratings
del movie_genres, user_genre_total, user_genre_count

## Actor and Director Ratings Summary

In [22]:
# Create movies dataframe subset that only contains the actors or directors
actor_cols = [mg for mg in list(movies_df_rated.columns) if mg[0:6] == 'actor_']
director_cols = [mg for mg in list(movies_df_rated.columns) if mg[0:9] == 'director_']
people_cols = actor_cols + director_cols
movie_people = movies_df_rated[people_cols]
# # Cross the binarized ratings dataframe with the genres dataframe 
# #   - this is now a dataframe that contains counts of rated movies by genre for each user
user_movie_people_count = ratings_spread_binary.dot(movie_people)

In [23]:
# For testing purposes, create list of Alfred Hitchcock movies (can be done for any director)
hitchcock_movies = list(movies_df_rated.item_id.iloc[np.where(movies_df_rated.director_Alfred_Hitchcock == 1)])
print(hitchcock_movies)

['item_id_1219', 'item_id_2176', 'item_id_2177', 'item_id_2178', 'item_id_2179', 'item_id_2180', 'item_id_2181', 'item_id_2182', 'item_id_2183', 'item_id_2184', 'item_id_2185', 'item_id_2186', 'item_id_2187', 'item_id_2200', 'item_id_2201', 'item_id_2202', 'item_id_2203', 'item_id_1086', 'item_id_1333', 'item_id_965', 'item_id_903', 'item_id_904', 'item_id_908', 'item_id_928', 'item_id_929', 'item_id_930', 'item_id_931', 'item_id_933', 'item_id_2204', 'item_id_2205', 'item_id_2206', 'item_id_2207', 'item_id_2208', 'item_id_2209', 'item_id_2210', 'item_id_2211', 'item_id_2212', 'item_id_2213', 'item_id_2214', 'item_id_2215', 'item_id_2216', 'item_id_2217', 'item_id_2218', 'item_id_2219', 'item_id_2220', 'item_id_2221', 'item_id_2222', 'item_id_2223', 'item_id_2224', 'item_id_2225', 'item_id_2226', 'item_id_2227', 'item_id_2229']


In [24]:
# Determine how many ratings were made for Hitchcock movies
ratings_df_temp = ratings_df.iloc[np.where([item in hitchcock_movies for item in ratings_df.item_id])]
print 'number of ratings:', len(ratings_df_temp)

number of ratings: 133947


In [25]:
# Make sure that the ratings were spread correctly and that the number of ratings remains constant
print 'number of ratings:', np.sum(np.sum(ratings_spread_binary[hitchcock_movies]))

number of ratings: 133947.0


In [26]:
# Confirm that the dot product maintained a correct number of ratings
print 'number of ratings:', np.sum(user_movie_people_count.director_Alfred_Hitchcock)

number of ratings: 133947.0


In [27]:
actor_views = user_movie_people_count[actor_cols].apply(np.sum, axis = 0)
# This total will be much greater than the total number of ratings because an individual movie has multiple actors
print 'total actor views: ', np.sum(actor_views)
director_views = user_movie_people_count[director_cols].apply(np.sum, axis = 0)
# This total may be greater than the total number of ratings if one movie had multiple directors
print 'total director views: ', np.sum(director_views)

total actor views:  79697734.0
total director views:  22443503.0


In [28]:
del movie_people, user_movie_people_count, hitchcock_movies, ratings_df_temp

In [29]:
# Filter actors to top 200, or ~7800, based on number of views
top_200_actors = actor_views.sort_values(ascending=False).head(200)
top_200_actors_names = top_200_actors.index
top_200_actors

actor_Harrison_Ford            428106.0
actor_Tom_Hanks                427393.0
actor_Bruce_Willis             305269.0
actor_Jim_Carrey               281397.0
actor_Robert_De_Niro           278569.0
actor_Tom_Cruise               273368.0
actor_Arnold_Schwarzenegger    263610.0
actor_Gene_Hackman             235619.0
actor_Jack_Nicholson           221905.0
actor_Sean_Connery             217081.0
actor_Robin_Williams           213372.0
actor_John_Travolta            212490.0
actor_Johnny_Depp              205983.0
actor_Morgan_Freeman           202681.0
actor_Al_Pacino                196081.0
actor_Mel_Gibson               194537.0
actor_Sigourney_Weaver         192514.0
actor_Tommy_Lee_Jones          191835.0
actor_Kevin_Spacey             184259.0
actor_Nicolas_Cage             183530.0
actor_Brad_Pitt                183302.0
actor_Keanu_Reeves             181029.0
actor_Sandra_Bullock           180589.0
actor_Carrie_Fisher            174954.0
actor_Val_Kilmer               172634.0


In [30]:
# Filter directors to top 50, of ~2200, based on number of views
top_50_directors = director_views.sort_values(ascending=False).head(50)
top_50_directors_names = top_50_directors.index
top_50_directors

director_Steven_Spielberg        428364.0
director_Robert_Zemeckis         234945.0
director_James_Cameron           215943.0
director_Tim_Burton              193414.0
director_Quentin_Tarantino       187630.0
director_Ron_Howard              163861.0
director_Ridley_Scott            158264.0
director_Joel_Coen               154372.0
director_Ethan_Coen              154372.0
director_Stanley_Kubrick         149729.0
director_Martin_Scorsese         147813.0
director_Rob_Reiner              142527.0
director_John_McTiernan          136972.0
director_Francis_Ford_Coppola    134652.0
director_Peter_Jackson           134111.0
director_Alfred_Hitchcock        133947.0
director_Terry_Gilliam           132581.0
director_Richard_Donner          131463.0
director_Chris_Columbus          124617.0
director_George_Lucas            121443.0
director_Roland_Emmerich         113935.0
director_Barry_Sonnenfeld        110302.0
director_Christopher_Nolan       108897.0
director_Joel_Schumacher         1

In [31]:
# Filter out actors and directors not in the top 200 and 50
# Create movies dataframe subset that only contains the top actors and directors
movie_people_top = movies_df_rated[list(top_200_actors_names) + list(top_50_directors_names)]
# Cross the new ratings dataframe with the  movie_people dataframes
#   - this is now a datframe with sum of ratings by actor and director for each user
user_movie_people_total = ratings_spread.dot(movie_people_top)
# Cross the binarized ratings dataframe with the movie_people dataframes
#   - this is now a dataframe with counts of rated movies by actor and director for each user
user_movie_people_count = ratings_spread_binary.dot(movie_people_top)

In [32]:
user_movie_people_avg_ratings = user_movie_people_total / user_movie_people_count
user_movie_people_avg_ratings = user_movie_people_avg_ratings.add_suffix('_avg_rating').reset_index()

In [33]:
# users_metadata = pd.merge(users_metadata, user_movie_people_avg_ratings, on = 'user_id')
users_metadata = pd.merge(user_genre_avg_ratings, user_movie_people_avg_ratings, on = 'user_id')
users_metadata.head()

,user_id,ml_genre_Action_avg_rating,ml_genre_Adventure_avg_rating,ml_genre_Animation_avg_rating,ml_genre_Children_avg_rating,ml_genre_Comedy_avg_rating,ml_genre_Crime_avg_rating,ml_genre_Documentary_avg_rating,ml_genre_Drama_avg_rating,ml_genre_Fantasy_avg_rating,...,director_Clint_Eastwood_avg_rating,director_Frank_Darabont_avg_rating,director_Wilfred_Jackson_avg_rating,director_Jan_de_Bont_avg_rating,director_Peter_Farrelly_avg_rating,director_Bobby_Farrelly_avg_rating,director_Oliver_Stone_avg_rating,director_John_Landis_avg_rating,director_Jay_Roach_avg_rating,director_Sam_Raimi_avg_rating
0,user_id_1,3.727273,3.787671,3.650000,3.605263,3.731707,3.809524,NaN,3.767442,3.789855,...,4.0,4.0,NaN,NaN,NaN,NaN,4.0,4.0,3.5,3.833333
1,user_id_10,3.812500,3.909091,4.000000,4.000000,4.000000,4.250000,NaN,3.840000,4.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,user_id_100,3.500000,3.500000,4.000000,2.500000,3.500000,3.800000,4.0,3.538462,3.666667,...,NaN,3.0,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN
3,user_id_1000,4.328947,4.239130,4.100000,3.900000,3.500000,4.500000,NaN,4.500000,4.125000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,user_id_10000,4.400000,4.705882,4.428571,4.307692,4.466667,4.250000,NaN,4.521739,3.875000,...,NaN,NaN,4.0,4.5,5.0,5.0,NaN,NaN,NaN,NaN


In [38]:
del movie_people_top, user_movie_people_total, user_movie_people_count, \
    user_movie_people_avg_ratings, ratings_spread, ratings_spread_binary, \
    top_200_actors, top_50_directors, ratings_df, actor_views, \
    director_views

## Convert to Spark Dataframe and Save

In [39]:
users_metadata['user_id'] = users_metadata.user_id.apply(lambda x: int(x[8:]))
sorted_columns = list(users_metadata.columns.sort_values())
users_metadata = users_metadata[sorted_columns]
users_metadata_spark = sqlContext.createDataFrame(users_metadata)
users_metadata_spark.write.format('parquet').mode('overwrite').save(data_path + 'users_metadata_20m')

In [40]:
movie_cols_to_keep = ['item_id', 'title', 'imdb_id', 'imdb_rating', 
                       'imdb_votes', 'metascore', 'runtime', 'year'] +\
        genre_cols + list(top_50_directors_names) + list(top_200_actors_names)
movies_df_rated['item_id'] = movies_df_rated.item_id.apply(lambda x: int(x[8:]))
movie_metadata_ohe_subset = sqlContext.createDataFrame(movies_df_rated[movie_cols_to_keep])
movie_metadata_ohe_subset.write.format('parquet').mode('overwrite').save(data_path + 'movie_20m_metadata_OHE_subset')

In [ ]:
# genre_and_people_cols = ['item_id'] + genre_cols + list(top_50_directors_names) + list(top_200_actors_names)
# movie_genre_and_people_metadata_ohe_subset = movies_metadata[genre_and_people_cols]
# movie_genre_and_people_metadata_ohe_subset.write.format('parquet').mode('overwrite'). \
#         save(data_path + 'movie_20m_genre_and_people_metadata_ohe_subset')

In [41]:
sc.stop()